In [ ]:
%matplotlib notebook
%precision 3

from IPython.display import set_matplotlib_formats
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

import matplotlib
import math
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [9.5, 7]
import pandas
from random import shuffle
from scipy.stats import poisson
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
# set this to where you placed https://berthub.eu/antonie/gcskewdb.csv.gz and unpacked https://berthub.eu/antonie/gcfits.tar.bz2
prefix="/home/ahu/git/antonie/genomes/ncbi-genomes-2021-08-19/"
m=pandas.read_csv(prefix+"gcskewdb.csv")
m

In [ ]:
chromoname="NZ_CP019870.1" # c difficile
fitted = pandas.read_csv(prefix+"/"+chromoname+"_fit.csv")

us=m[m.name==chromoname]

plt.figure()
plt.plot(fitted.pos, fitted.gcskew, label="Cumulative GC skew")
plt.plot(fitted.pos, fitted.predgcskew, label="Fitted GC skew")
#plt.plot(fitted.pos, fitted.taskew, label="Cumulative TA skew")
#plt.plot(fitted.pos, fitted.predtaskew, label="Fitted TA skew")

#plt.plot(fitted.pos, fitted.sbskew, label="Cumulative SB skew")
#plt.plot(fitted.pos, fitted.predsbskew, label="Fitted SB skew")


leshift=us["shift"].item()
if leshift > 0:
    plt.axvline(leshift, ls=':', color='black')
else:
    plt.axvline(fitted.pos.max() + leshift, ls=':', color='black')

plt.xlabel("Locus")
plt.ylabel("Skew")
plt.legend()
plt.title(chromoname + " alpha1 " + str(round(us["alpha1gc"].item(),3)) + " alpha2 " + str(round(us["alpha2gc"].item(),3)) + 
          " div " + str(round(us["div"].item(),3)))
plt.grid()


In [ ]:
plt.figure()

leX=fitted.sbskew.diff().rolling(10, center=True).mean()
leY=fitted.gcskew.diff().rolling(10, center=True).mean()


frame = { 'predleading': fitted.predleading, 'x': leX, 'y': leY }
result = pandas.DataFrame(frame)
#print(result.tail(20))


sub=result[(result.predleading==1) & ((result.x < 0) | (result.x>0)) & ((result.y < 0) | (result.y > 0)) ]
#z = np.polyfit(sub.x, sub.y, 1)
#p = np.poly1d(z)
#xp = np.linspace(leX.min(), leX.max(), 100)
#plt.plot(xp, p(xp), ':', color='red')

model = smf.quantreg('y ~ x', sub).fit(q=0.5)
print(model.summary())

print(model.params['Intercept'], model.params['x'])

get_y = lambda a, b: a + b * sub.x
y = get_y(model.params['Intercept'], model.params['x'])
plt.plot(sub.x, y, color='black')


sub=result[(result.predleading==0) & ((result.x < 0) | (result.x>0)) & ((result.y < 0) | (result.y > 0))]

#z = np.polyfit(sub.x, sub.y, 1)
#p = np.poly1d(z)
#xp = np.linspace(leX.min(), leX.max(), 100)
#plt.plot(xp, p(xp), ':', color='red')

model = smf.quantreg('y ~ x', sub).fit(q=0.5)
print(model.summary())
print(model.params['Intercept'], model.params['x'])
get_y = lambda a, b: a + b * sub.x
y = get_y(model.params['Intercept'], model.params['x'])
plt.plot(sub.x, y, color='black')


plt.scatter(result[result.predleading==0].x, result[result.predleading==0].y, marker="+", color='blue', label="GC lagging")
plt.scatter(result[result.predleading==1].x, result[result.predleading==1].y, marker="^", color='blue', label="GC leading")

leX=fitted.sbskew.diff().rolling(10, center=True).mean()
leY=fitted.taskew.diff().rolling(10, center=True).mean()

z = np.polyfit(leX.dropna(), leY.dropna(), 1)
p = np.poly1d(z)
xp = np.linspace(leX.min(), leX.max(), 100)
plt.plot(xp, p(xp), ':', color='red')

plt.scatter(leX, leY,  s=4, label="TA", color='orange')
plt.xlabel("SB skew")
plt.ylabel("GC/TA skew")
plt.title(chromoname)
plt.legend()


plt.grid()

In [ ]:
# find and plot a bunch of flat chromosomes

sel=m[(m.rmsGC < 0.0020) & (m["alpha1gc"] < 0.0018) & (m["alpha1gc"] > 0)]  #  & (m.realm3=="Firmicutes")
#sel=m[(m["div"] < 0.22) & (m.rmsGC < 0.002) ]
d=4
fig,axs=plt.subplots(d,d, sharex=False, sharey=False, constrained_layout=True)
a=0
b=0
names=sel.name.unique()
print(len(names))
shuffle(names)
for k in names:
        fitted = pandas.read_csv(prefix+"/"+k+"_fit.csv")
        t=m[m.name==k]
        axs[b,a].plot(fitted.pos, fitted.gcskew)
        axs[b,a].plot(fitted.pos, fitted.predgcskew)
        #axs[b,a].plot(fitted.pos, fitted.taskew)
        #axs[b,a].plot(fitted.pos, fitted.predtaskew)

        #axs[b,a].get_yaxis().set_ticks([])
        axs[b,a].set_title(k, fontsize=8) #  + " "+str(1000*results[results.name==k].rmsGC.mean()), 
        #axs[b,a].grid()
        a=a+1
        if(a>=d):
            a=0
            b=b+1
        if(b == d):
            break
fig.suptitle("GC/TA skew in random bacterial chromosomes + fit")

In [ ]:
# Plot relation of GC and TA skew for Firmicutes and non-Firmicutes
plt.figure()
sel=m[m.realm3=="Firmicutes"]
plt.scatter(sel.alpha1gc, sel.alpha1ta, s=2, alpha=0.3, label="Firmicutes")
sel=m[m.realm3!="Firmicutes"]
plt.scatter(sel.alpha1gc, sel.alpha1ta, s=2, alpha=0.3, label="Non-Firmicutes")

plt.xlabel("alpha1 of GC skew")
plt.ylabel("alpha1 of TA skew")
plt.legend()
plt.title("Scatter plot of GC and TA skew")
plt.grid()

In [ ]:
# Plot prediction of GC/TA skew based on strand and codon bias and percentage non-coding
firmi=m[(m.realm3=="Firmicutes") & (m.rmsGC < 0.02) & (m.rmsTA < 0.02)]
plt.figure()
leX =  -(firmi.cfrac - firmi.gfrac)*(1-firmi.ngcount/firmi.siz)*firmi.alpha2sb

leY=firmi.alpha1gc

frame = {  'x': leX, 'y': leY }
sub = pandas.DataFrame(frame)

model = smf.quantreg('y ~ x', sub).fit(q=0.5)
print(model.summary())

print(model.params['Intercept'], model.params['x'])

get_y = lambda a, b: a + b * sub.x
y = get_y(model.params['Intercept'], model.params['x'])

plt.scatter(leX, leY, s=2, label="GC")
plt.plot(sub.x, y, color='black')

#

leX =  -(firmi.tfrac - firmi.afrac)*(1-firmi.ngcount/firmi.siz)*firmi.alpha2sb
leY=firmi.alpha1ta

frame = {  'x': leX, 'y': leY }
sub = pandas.DataFrame(frame)

model = smf.quantreg('y ~ x', sub).fit(q=0.5)
print(model.summary())

print(model.params['Intercept'], model.params['x'])

get_y = lambda a, b: a + b * sub.x
y = get_y(model.params['Intercept'], model.params['x'])
plt.scatter(leX, leY, s=2, label="TA")

plt.plot(sub.x, y, color='black', label='fit')


plt.grid()
#plt.scatter(firmi.ccounts2-firmi.tcounts2, firmi.alpha1gc)
plt.legend()
plt.xlabel("Product of gene strand bias, codon bias skew, percentage coding")
plt.ylabel("GC/TA skew fraction")
plt.title("Data for "+str(len(firmi))+ " Firmicute chromosomes")